<a href="https://colab.research.google.com/github/HanhaodiZhang/HATE/blob/main/mlm%2B%3C%3E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install -qq transformers

In [10]:
import torch
import numpy as np
import pandas as pd

In [11]:
df = pd.read_csv("olid-training.tsv", sep='\t', header=0, index_col='id')

df.head()

,tweet,subtask_a,subtask_b,subtask_c
id,,,,
86426,@USER She should ask a few native Americans wh...,OFF,UNT,NaN
90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,TIN,IND
16820,Amazon is investigating Chinese employees who ...,NOT,NaN,NaN
62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,UNT,NaN
43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,NaN,NaN


In [12]:
df_test_label = pd.read_csv("labels-levela.csv",header=None,names=['id','subtask_a'],index_col='id')
df_test_tweet = pd.read_csv("testset-levela.tsv", sep='\t', header=0, index_col='id')
df_test = pd.merge(df_test_tweet, df_test_label, on=['id'])
df_test.head()

,tweet,subtask_a
id,,
15923,#WhoIsQ #WheresTheServer #DumpNike #DECLASFISA...,OFF
27014,"#ConstitutionDay is revered by Conservatives, ...",NOT
30530,#FOXNews #NRA #MAGA #POTUS #TRUMP #2ndAmendmen...,NOT
13876,#Watching #Boomer getting the news that she is...,NOT
60133,#NoPasaran: Unity demo to oppose the far-right...,OFF


In [13]:
df['comment']='<'+df['subtask_a']+'> '+df['tweet']
df_test['comment'] = '<'+df_test['subtask_a']+'> '+df_test['tweet']

In [14]:
df.head()

,tweet,subtask_a,subtask_b,subtask_c,comment
id,,,,,
86426,@USER She should ask a few native Americans wh...,OFF,UNT,NaN,<OFF> @USER She should ask a few native Americ...
90194,@USER @USER Go home you’re drunk!!! @USER #MAG...,OFF,TIN,IND,<OFF> @USER @USER Go home you’re drunk!!! @USE...
16820,Amazon is investigating Chinese employees who ...,NOT,NaN,NaN,<NOT> Amazon is investigating Chinese employee...
62688,"@USER Someone should'veTaken"" this piece of sh...",OFF,UNT,NaN,"<OFF> @USER Someone should'veTaken"" this piece..."
43605,@USER @USER Obama wanted liberals &amp; illega...,NOT,NaN,NaN,<NOT> @USER @USER Obama wanted liberals &amp; ...


In [15]:

# M = df['comment']
 
# M.to_csv('train.txt',header=False,index=False)
# M = df_test['comment']
# M.to_csv('val.txt', header=False, index=False)

with open("train.txt", 'w') as f:
    for line in df['comment'].to_list():
        f.write(line + '\n')

with open("val.txt", 'w') as f:
    for line in df_test['comment'].to_list():
        f.write(line + '\n')

In [16]:
from transformers import BertTokenizer

PRE_TRAINED_MODEL_NAME = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)
tokenizer.add_special_tokens({'additional_special_tokens':["<OFF>","<NOT>"]})

2

In [17]:
from transformers import BertForMaskedLM

model = BertForMaskedLM.from_pretrained('bert-base-uncased')
model.resize_token_embeddings(len(tokenizer))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Embedding(30524, 768)

In [18]:
# TRAIN ONLY
from transformers import LineByLineTextDataset
from transformers import DataCollatorForLanguageModeling

train_dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="train.txt",
    block_size=128,
)

valid_dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="val.txt",
    block_size=128,
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

/usr/local/lib/python3.6/dist-packages/transformers/data/datasets/language_modeling.py:128: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_mlm.py
  FutureWarning,


In [19]:
# TRAIN ONLY
from transformers import Trainer, TrainingArguments, EvalPrediction

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.squeeze(preds) if is_regression else np.argmax(preds, axis=1)
    
    return {"accuracy": (preds == p.label_ids).astype(np.float32).mean().item()}

model.cuda()
training_args = TrainingArguments(
    output_dir="./mlm",
    overwrite_output_dir=True,
    num_train_epochs=10,
    per_device_train_batch_size=16,
    save_steps=1000,
    save_total_limit=10,
    evaluation_strategy='epoch',
    learning_rate=2e-4,
    weight_decay=1e-5,
    lr_scheduler_type='cosine',
    warmup_steps=500,
    metric_for_best_model='eval_loss',
    load_best_model_at_end=True,
    disable_tqdm=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    # compute_metrics=compute_metrics
    # prediction_loss_only=True,
)

In [20]:
%%time
# TRAIN ONLY
trainer.train()

{'loss': 2.0544, 'learning_rate': 0.0002, 'epoch': 0.6}
{'eval_loss': 2.072282552719116, 'eval_runtime': 1.7421, 'eval_samples_per_second': 493.645, 'epoch': 1.0}
{'loss': 1.2789, 'learning_rate': 0.0001979686968042461, 'epoch': 1.21}
{'loss': 1.256, 'learning_rate': 0.00019195731107044594, 'epoch': 1.81}
{'eval_loss': 2.1243677139282227, 'eval_runtime': 1.7399, 'eval_samples_per_second': 494.285, 'epoch': 2.0}
{'loss': 1.2285, 'learning_rate': 0.00018221006173963912, 'epoch': 2.42}
{'eval_loss': 2.151510715484619, 'eval_runtime': 1.7455, 'eval_samples_per_second': 492.705, 'epoch': 3.0}
{'loss': 1.2224, 'learning_rate': 0.00016912294118613517, 'epoch': 3.02}
{'loss': 1.1771, 'learning_rate': 0.0001532276276060051, 'epoch': 3.62}
{'eval_loss': 2.0780997276306152, 'eval_runtime': 1.7564, 'eval_samples_per_second': 489.639, 'epoch': 4.0}
{'loss': 1.1193, 'learning_rate': 0.0001351698850247055, 'epoch': 4.23}
{'loss': 1.0931, 'learning_rate': 0.00011568332844650622, 'epoch': 4.83}
{'eval_

TrainOutput(global_step=8280, training_loss=1.092704625521305, metrics={'train_runtime': 1159.9963, 'train_samples_per_second': 7.138, 'epoch': 10.0})

In [21]:
trainer.save_model("./mlm")

In [22]:

from transformers import pipeline
import pandas as pd
from tqdm.notebook import tqdm
from transformers import BertTokenizer

PRE_TRAINED_MODEL_NAME = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)
tokenizer.add_special_tokens({'additional_special_tokens':["<OFF>","<NOT>"]})

fill_mask = pipeline(
    "fill-mask",
    model="./mlm",
    tokenizer=tokenizer
)

Some weights of BertModel were not initialized from the model checkpoint at ./mlm and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
fill_mask('<NOT> @USER @USER You are [MASK].')

[{'score': 0.24912793934345245,
  'sequence': '@ user @ user you are welcome.',
  'token': 6160,
  'token_str': 'w e l c o m e'},
 {'score': 0.11235679686069489,
  'sequence': '@ user @ user you are correct.',
  'token': 6149,
  'token_str': 'c o r r e c t'},
 {'score': 0.0975445955991745,
  'sequence': '@ user @ user you are right.',
  'token': 2157,
  'token_str': 'r i g h t'},
 {'score': 0.059810008853673935,
  'sequence': '@ user @ user you are beautiful.',
  'token': 3376,
  'token_str': 'b e a u t i f u l'},
 {'score': 0.051674071699380875,
  'sequence': '@ user @ user you are amazing.',
  'token': 6429,
  'token_str': 'a m a z i n g'}]

In [24]:
fill_mask('<OFF> @USER @USER You are [MASK].')

[{'score': 0.236549973487854,
  'sequence': '@ user @ user you are disgusting.',
  'token': 19424,
  'token_str': 'd i s g u s t i n g'},
 {'score': 0.20990410447120667,
  'sequence': '@ user @ user you are pathetic.',
  'token': 17203,
  'token_str': 'p a t h e t i c'},
 {'score': 0.031589407473802567,
  'sequence': '@ user @ user you are sick.',
  'token': 5305,
  'token_str': 's i c k'},
 {'score': 0.02628418244421482,
  'sequence': '@ user @ user you are insane.',
  'token': 9577,
  'token_str': 'i n s a n e'},
 {'score': 0.022228850051760674,
  'sequence': '@ user @ user you are hilarious.',
  'token': 26316,
  'token_str': 'h i l a r i o u s'}]

In [25]:
df = pd.read_csv("olid-training.tsv", sep='\t', header=0, index_col='id')

In [26]:
N = 1
K = 2

In [31]:
import spacy
import random
random.seed(17)

# "python -m spacy download en" --> install spacy model
nlp = spacy.load("en") 
  
# randomly mask noun
def mask_noun_and_adj(text, n=1):
    doc = nlp(text)
    noun_adj_position = []

    words = []
    # get all noun and adj indices
    for i, token in enumerate(doc):
        words.append(token.text_with_ws)
        if (token.pos_ == "NOUN" or token.pos_ == "ADJ") and token.text != "#":
            noun_adj_position.append(i)
        
    if n > len(words):
        # mask all words if n > sentence length
        mask_position = list(range(len(words)))
    elif n > len(noun_adj_position):
        # mask all nouns and adjs and randomly pick the rest
        mask_position = noun_adj_position
        rest = [i for i in range(len(words)) if i not in noun_adj_position]
        rest = random.sample(rest, n - len(noun_adj_position))
        mask_position += rest
    else:
        mask_position = random.sample(noun_adj_position, n)
    mask_word=[]
    for idx in mask_position:
        mask_word.append(words[idx])
        if words[idx].startswith(' ') and words[idx].endswith(' '):
            words[idx] = ' [MASK] '
        elif words[idx].startswith(' '):
            words[idx] = ' [MASK]'
        elif words[idx].endswith(' '):
            words[idx] = '[MASK] '
        else:
            words[idx] = '[MASK]'
        
    return "".join(words), mask_word

In [32]:
mask_noun_and_adj('@ user @ user you are insane.', n=1)

('@ user @ user you are [MASK].', ['insane'])

In [70]:
result=[]
maskwords=["insane"]
augment_off_list = fill_mask('@ user @ user you are [MASK]')
for s in augment_off_list:
      if s['token_str']=='[ U N K ]':
        continue
      try:
        temp = '@ user @ user you are insane'.split()
        tweet=''
        tweet_list=[]
        for word in temp:
          if word == maskwords[0]:
            word= s['token_str'].replace(' ', '')
          tweet_list.append(word)
        tweet = " ".join(tweet_list)   
        result.append(tweet)
      except IndexError as error:
        pass

In [71]:
result

['@ user @ user you are right',
 '@ user @ user you are beautiful',
 '@ user @ user you are correct',
 '@ user @ user you are .']

In [61]:

list('r i g h t')

['r', ' ', 'i', ' ', 'g', ' ', 'h', ' ', 't']

In [74]:
df_aug = pd.DataFrame({
    'id':[],
    'tweet':[],
    'subtask_a':[],
})
df_aug = df_aug.set_index('id')

In [75]:
for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    masked_tweet, maskwords = mask_noun_and_adj(row['tweet'], N)

    masked_tweet_with_off = '<OFF> ' + masked_tweet
    masked_tweet_with_not = '<NOT> ' + masked_tweet

    augment_off_list = fill_mask(masked_tweet_with_off)
    augment_not_list = fill_mask(masked_tweet_with_not)
    aug_tweets=[]
    aug_tweets_list = augment_off_list + augment_not_list
    for s in aug_tweets_list:
      if s['token_str']=='[ U N K ]':
        continue
      try:
        temp = row['tweet'].split()
        tweet=''
        tweet_list=[]
        for word in temp:
          if word == maskwords[0]:
            word= s['token_str'].replace(' ', '')
          tweet_list.append(word)
        tweet = " ".join(tweet_list)   
        aug_tweets.append(tweet)
      except IndexError as error:
        pass
    aug_labels = ['OFF'] * len(augment_off_list) + ['NOT'] * len(augment_not_list)

    # randomly sample from the mixture of augmentation with both OFF and NOT
    sample_idxs = random.sample(range(len(aug_tweets)), K)

    tweets = [aug_tweets[i] for i in sample_idxs]
    labels = [aug_labels[i] for i in sample_idxs]
    for i, (tweet, label) in enumerate(zip(tweets, labels)):
        id = int(index*10 + i + 1)
        df_aug = df_aug.append([{'id':id,'tweet':tweet,'subtask_a':label}],ignore_index=False)

In [76]:
df_aug = df_aug.set_index('id')
df_aug.to_csv("bert_aug_N={}_k={}.csv".format(N,K))